In [19]:
%matplotlib inline
%load_ext autoreload 
%autoreload 2

from __future__ import division

import numpy as np
import os
import mne
import pandas as pd
import matplotlib.pyplot as plt

from scipy import io

from data_utils import *
from preprocessing import *

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score

import xgboost as xgb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
data = standard_import(nstride=50, filt='letter')
dim(data)           


Converting time column to int64...
2 1 3 7701 64 400 


In [10]:
X, Y = pipeline_standard(data[0]), data[1]

dim(X)
dim(Y)

1 3 7701 2560 
1 3 7701 


In [11]:
X0, Y0 = X[0], Y[0]

X_train, X_valid, X_test = X0
Y_train, Y_valid, Y_test = Y0

X_train, X_valid, X_test = map(np.array, (X_train, X_valid, X_test))
Y_train, Y_valid, Y_test = map(np.array, (Y_train, Y_valid, Y_test))

Y_train_, Y_valid_, Y_test_ = map(to_cat, (Y_train, Y_valid, Y_test))

/Users/Yannis/anaconda2/lib/python2.7/site-packages/keras/utils/np_utils.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y = np.zeros((len(y), nb_classes))
/Users/Yannis/anaconda2/lib/python2.7/site-packages/keras/utils/np_utils.py:16: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y[i, y[i]] = 1.


In [14]:
def create_model(input_len,output_len):

    model = Sequential()
    
    model.add(Dense(output_len, input_dim=input_len))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer="adadelta")

    return model


model = create_model(X_train.shape[-1], Y_train_.shape[-1])

stopper = EarlyStopping(monitor='val_loss', patience=4, verbose=0, mode='min')
model.fit(X_train, Y_train_, batch_size=128, nb_epoch=128, 
    verbose=0, validation_split=0.1, callbacks=[stopper])



In [15]:
Y_pred = model.predict_classes(X_test)

 960/1920 [==============>...............] - ETA: 0s

In [18]:
print accuracy_score(Y_test, Y_pred)

0.1421875


In [20]:
# create model
model = xgb.XGBClassifier()

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score

print accuracy_score(Y_true, Y_pred)

NameError: name 'Y_true' is not defined

In [21]:
print accuracy_score(Y_test, Y_pred)

0.155208333333


In [ ]:
import xgboost